In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from keras import models, layers, optimizers, regularizers
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:463: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:464: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:465: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([(

### From our work on the seattle data, we've determined what the best columns are. We'll keep those for this dataset

In [2]:
df = pd.read_csv('san_francisco_listings.csv')
df = df.rename(columns={'neighbourhood_cleansed':'neighbourhood_group'})
print(df.columns)
cols_to_keep = ['host_response_time', 'host_response_rate', 'host_has_profile_pic',
       'neighbourhood_group', 'zipcode', 'market', 'smart_location',
       'latitude', 'longitude', 'is_location_exact', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'price',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights', 'calendar_updated',
       'has_availability', 'availability_90', 'number_of_reviews', 'first_review', 'last_review',
       'requires_license', 'instant_bookable', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification', 'security_deposit', 'cleaning_fee']


df = df[cols_to_keep]

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       ...
       'instant_bookable', 'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object', length=106)


In [3]:
len(df.columns)


35

In [6]:
print(df.isna().sum())

host_response_time                   927
host_response_rate                   927
host_has_profile_pic                   8
neighbourhood_group                    0
zipcode                              245
market                                21
smart_location                         0
latitude                               0
longitude                              0
is_location_exact                      0
property_type                          0
room_type                              0
accommodates                           0
bathrooms                             12
bedrooms                               4
beds                                   9
bed_type                               0
price                                  0
guests_included                        0
extra_people                           0
minimum_nights                         0
maximum_nights                         0
calendar_updated                       0
has_availability                       0
availability_90 

In [7]:
useless = ['host_response_time', 'host_response_rate', 'zipcode', 
           'first_review', 'last_review', 'cleaning_fee']
df = df.drop(useless, axis=1)

In [8]:
df.columns

Index(['host_has_profile_pic', 'neighbourhood_group', 'market',
       'smart_location', 'latitude', 'longitude', 'is_location_exact',
       'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'bed_type', 'price', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'calendar_updated',
       'has_availability', 'availability_90', 'number_of_reviews',
       'requires_license', 'instant_bookable', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'security_deposit'],
      dtype='object')

### Now, we'll perform the same feature engineering and test the same neural networks to confirm neural nets

In [11]:
for col in ['bathrooms', 'bedrooms', 'beds']:
    df[col].fillna(df[col].mean(), inplace=True)

df.price = df.price.str[1:-3]
df.price = df.price.str.replace(",", "")
df.price = df.price.str.replace(".", "") # decimal at end
df.price = df.price.astype('int64')

df.extra_people = df.extra_people.str[1:-3]
df.extra_people = df.extra_people.str.replace(",", "")
df.extra_people = df.extra_people.str.replace(".", "")
df.extra_people = df.extra_people.astype('int64')

# df.cleaning_fee = df.cleaning_fee.str[1:-3]
# df.cleaning_fee = df.cleaning_fee.str.replace(",", "")
# df.cleaning_fee = df.cleaning_fee.str.replace(".", "")
# df.cleaning_fee = df.cleaning_fee.astype('float64')

<ipython-input-11-0b656e988400>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.price = df.price.str.replace(".", "") # decimal at end
<ipython-input-11-0b656e988400>:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.extra_people = df.extra_people.str.replace(".", "")


In [12]:
# for col in ['bathrooms', 'bedrooms', 'beds']:
#     df[col].fillna(df[col].mean(), inplace=True)
    
# df.price = df.price.str[1:-3]
# df.price = df.price.str.replace(",", "")
# df.price = df.price.str.replace(".", "") # decimal at end
# df.price = df.price.astype('int64')

# df.extra_people = df.extra_people.str[1:-3]
# df.extra_people = df.extra_people.str.replace(",", "")
# df.extra_people = df.extra_people.str.replace(".", "")
# df.extra_people = df.extra_people.astype('int64')

In [13]:
df['property_type'] = df.property_type.replace({
    'Townhouse': 'House',
    'Bungalow': 'House',
    'Loft': 'Apartment',
    'Chalet': 'Cabin',
    'Tiny house': 'House',
    'Earth house': 'House',
    'Condominium': 'Apartment',
    'Boutique hotel': 'Hotel',
    'Aparthotel': 'Hotel',
    'Serviced apartment': 'Apartment'
    })

In [14]:
transformed_df = pd.get_dummies(df)

# to_drop = ['beds',
#            'bedrooms',
#            'guests_included',
#            'room_type_Private room']
# to_drop.extend(list(transformed_df.columns[transformed_df.columns.str.endswith('nan')]))
# transformed_df = transformed_df.drop(to_drop, axis=1, inplace=False)

numerical_columns = ['accommodates', 'availability_90', 'bathrooms', 'extra_people',
                     'maximum_nights', 'minimum_nights', 'number_of_reviews',
                     'price']

numerical_columns = [i for i in numerical_columns if i not in ['availability_90']]# Removing items not to be transformed

for col in numerical_columns:
    transformed_df[col] = transformed_df[col].astype('float64').replace(0.0, 0.01) # Replacing 0s with 0.01 because log(0) undefined
    transformed_df[col] = np.log(transformed_df[col])

In [15]:
X = transformed_df.drop('price', axis=1)
y = transformed_df.price

scaler = StandardScaler()
print(X.columns)
X = pd.DataFrame(scaler.fit_transform(X), columns=list(X.columns))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

Index(['latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights',
       ...
       'security_deposit_$850.00 ', 'security_deposit_$899.00 ',
       'security_deposit_$900.00 ', 'security_deposit_$95.00 ',
       'security_deposit_$950.00 ', 'security_deposit_$960.00 ',
       'security_deposit_$975.00 ', 'security_deposit_$990.00 ',
       'security_deposit_$995.00 ', 'security_deposit_$999.00 '],
      dtype='object', length=266)


In [16]:
def nn_model_evaluation(model, skip_epochs=0, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test):
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    print("Training MSE:", round(mean_squared_error(y_train, y_train_pred),4))
    print("Validation MSE:", round(mean_squared_error(y_test, y_test_pred),4))
    print("\nTraining r2:", round(r2_score(y_train, y_train_pred),4))
    print("Validation r2:", round(r2_score(y_test, y_test_pred),4))

### Now, we can test the same neural networks to see what happens

In [17]:
def ModelResults(classifier):
    classifier.fit(X_train, y_train)
    train_predict = classifier.predict(X_train)
    test_predict = classifier.predict(X_test)
    print("\nR^2 Train:", r2_score(y_train, train_predict))
    print("R^2 Test:", r2_score(y_test, test_predict))
    
ETR = ExtraTreesRegressor(max_depth=17, n_estimators=100, n_jobs=-1)
RFR = RandomForestRegressor(max_depth=17, n_estimators=100, n_jobs=-1)
GBR = GradientBoostingRegressor(n_estimators=200, learning_rate=0.2)
ModelResults(ETR)
ModelResults(RFR)
ModelResults(GBR)


R^2 Train: 0.958037509857866
R^2 Test: 0.7191792936689214

R^2 Train: 0.9382484553733835
R^2 Test: 0.725731831965202

R^2 Train: 0.8061763128584237
R^2 Test: 0.7137513726471116


In [14]:
nn4 = models.Sequential()
nn4.add(layers.Dense(128, input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l1(0.01), activation='relu'))
nn4.add(layers.Dense(256, kernel_regularizer=regularizers.l1(0.01), activation='relu'))
nn4.add(layers.Dense(256, kernel_regularizer=regularizers.l1(0.01), activation='relu'))
nn4.add(layers.Dense(512, kernel_regularizer=regularizers.l1(0.01), activation='relu'))
nn4.add(layers.Dense(1, activation='linear'))

# Compiling the model
nn4.compile(loss='mean_squared_error',
            optimizer='SGD',
            metrics=['mean_squared_error'])

nn4_history = nn4.fit(X_train,
                  y_train,
                  epochs=200,
                  batch_size=256,
                  validation_split = 0.1)

nn_model_evaluation(nn4)

Train on 5839 samples, validate on 649 samples
Epoch 1/200
5839/5839 [==============================] - 1s 146us/step - loss: 137.9855 - mean_squared_error: 6.0543 - val_loss: 131.9889 - val_mean_squared_error: 3.2000
Epoch 2/200
5839/5839 [==============================] - 1s 86us/step - loss: 127.2457 - mean_squared_error: 1.2175 - val_loss: 124.0643 - val_mean_squared_error: 1.0970
Epoch 3/200
5839/5839 [==============================] - 1s 87us/step - loss: 120.8148 - mean_squared_error: 0.5618 - val_loss: 118.0452 - val_mean_squared_error: 0.7816
Epoch 4/200
5839/5839 [==============================] - 0s 83us/step - loss: 114.9913 - mean_squared_error: 0.3757 - val_loss: 112.3748 - val_mean_squared_error: 0.6704
Epoch 5/200
5839/5839 [==============================] - 0s 85us/step - loss: 109.4552 - mean_squared_error: 0.3308 - val_loss: 106.8600 - val_mean_squared_error: 0.5719
Epoch 6/200
5839/5839 [==============================] - 0s 82us/step - loss: 104.0580 - mean_squared_

5839/5839 [==============================] - 1s 116us/step - loss: 0.4684 - mean_squared_error: 0.2142 - val_loss: 0.4961 - val_mean_squared_error: 0.2425
Epoch 99/200
5839/5839 [==============================] - 1s 134us/step - loss: 0.4661 - mean_squared_error: 0.2129 - val_loss: 0.4949 - val_mean_squared_error: 0.2429
Epoch 100/200
5839/5839 [==============================] - 1s 89us/step - loss: 0.4654 - mean_squared_error: 0.2135 - val_loss: 0.4940 - val_mean_squared_error: 0.2431
Epoch 101/200
5839/5839 [==============================] - 0s 85us/step - loss: 0.4641 - mean_squared_error: 0.2135 - val_loss: 0.4927 - val_mean_squared_error: 0.2422
Epoch 102/200
5839/5839 [==============================] - 1s 130us/step - loss: 0.4631 - mean_squared_error: 0.2134 - val_loss: 0.4958 - val_mean_squared_error: 0.2453
Epoch 103/200
5839/5839 [==============================] - 1s 91us/step - loss: 0.4621 - mean_squared_error: 0.2133 - val_loss: 0.4916 - val_mean_squared_error: 0.2441
Epoc

Epoch 147/200
5839/5839 [==============================] - 1s 103us/step - loss: 0.4356 - mean_squared_error: 0.2122 - val_loss: 0.4632 - val_mean_squared_error: 0.2396
Epoch 148/200
5839/5839 [==============================] - 1s 119us/step - loss: 0.4348 - mean_squared_error: 0.2119 - val_loss: 0.4651 - val_mean_squared_error: 0.2411
Epoch 149/200
5839/5839 [==============================] - 1s 118us/step - loss: 0.4353 - mean_squared_error: 0.2126 - val_loss: 0.4648 - val_mean_squared_error: 0.2426
Epoch 150/200
5839/5839 [==============================] - 1s 118us/step - loss: 0.4346 - mean_squared_error: 0.2124 - val_loss: 0.4658 - val_mean_squared_error: 0.2440
Epoch 151/200
5839/5839 [==============================] - 1s 118us/step - loss: 0.4346 - mean_squared_error: 0.2124 - val_loss: 0.4634 - val_mean_squared_error: 0.2419
Epoch 152/200
5839/5839 [==============================] - 1s 123us/step - loss: 0.4337 - mean_squared_error: 0.2121 - val_loss: 0.4638 - val_mean_squared_

Epoch 196/200
5839/5839 [==============================] - 1s 87us/step - loss: 0.4252 - mean_squared_error: 0.2124 - val_loss: 0.4583 - val_mean_squared_error: 0.2457
Epoch 197/200
5839/5839 [==============================] - 1s 88us/step - loss: 0.4263 - mean_squared_error: 0.2135 - val_loss: 0.4535 - val_mean_squared_error: 0.2408
Epoch 198/200
5839/5839 [==============================] - 1s 88us/step - loss: 0.4248 - mean_squared_error: 0.2123 - val_loss: 0.4525 - val_mean_squared_error: 0.2401
Epoch 199/200
5839/5839 [==============================] - 1s 86us/step - loss: 0.4246 - mean_squared_error: 0.2123 - val_loss: 0.4561 - val_mean_squared_error: 0.2429
Epoch 200/200
5839/5839 [==============================] - 1s 87us/step - loss: 0.4247 - mean_squared_error: 0.2125 - val_loss: 0.4572 - val_mean_squared_error: 0.2446
Training MSE: 0.2115
Validation MSE: 0.1803

Training r2: 0.6075
Validation r2: 0.6395


In [11]:
nn4 = models.Sequential()
nn4.add(layers.Dense(128, input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l1(0.01), activation='relu'))
nn4.add(layers.Dense(256, kernel_regularizer=regularizers.l1(0.01), activation='relu'))
nn4.add(layers.Dense(512, kernel_regularizer=regularizers.l1(0.01), activation='relu'))
nn4.add(layers.Dense(1, activation='relu'))

# Compiling the model
nn4.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['mean_squared_error'])

nn4_history = nn4.fit(X_train,
                  y_train,
                  epochs=150,
                  batch_size=256,
                  validation_split = 0.1)

nn_model_evaluation(nn4)

Train on 5839 samples, validate on 649 samples
Epoch 1/150
5839/5839 [==============================] - 1s 119us/step - loss: 97.2004 - mean_squared_error: 8.6251 - val_loss: 81.0111 - val_mean_squared_error: 3.6832
Epoch 2/150
5839/5839 [==============================] - 0s 69us/step - loss: 69.8499 - mean_squared_error: 2.2149 - val_loss: 58.6920 - val_mean_squared_error: 1.2966
Epoch 3/150
5839/5839 [==============================] - 0s 68us/step - loss: 49.9368 - mean_squared_error: 0.7728 - val_loss: 41.4387 - val_mean_squared_error: 0.6272
Epoch 4/150
5839/5839 [==============================] - 0s 74us/step - loss: 35.2209 - mean_squared_error: 0.4096 - val_loss: 29.5018 - val_mean_squared_error: 0.4460
Epoch 5/150
5839/5839 [==============================] - 0s 69us/step - loss: 25.3989 - mean_squared_error: 0.3208 - val_loss: 21.6059 - val_mean_squared_error: 0.3887
Epoch 6/150
5839/5839 [==============================] - 0s 75us/step - loss: 18.7846 - mean_squared_error: 0.27

Epoch 50/150
5839/5839 [==============================] - 0s 81us/step - loss: 0.5124 - mean_squared_error: 0.2058 - val_loss: 0.5409 - val_mean_squared_error: 0.2351
Epoch 51/150
5839/5839 [==============================] - 0s 78us/step - loss: 0.5124 - mean_squared_error: 0.2076 - val_loss: 0.5385 - val_mean_squared_error: 0.2346
Epoch 52/150
5839/5839 [==============================] - 1s 94us/step - loss: 0.5110 - mean_squared_error: 0.2065 - val_loss: 0.5442 - val_mean_squared_error: 0.2369
Epoch 53/150
5839/5839 [==============================] - 0s 73us/step - loss: 0.5094 - mean_squared_error: 0.2059 - val_loss: 0.5419 - val_mean_squared_error: 0.2388
Epoch 54/150
5839/5839 [==============================] - 0s 81us/step - loss: 0.5119 - mean_squared_error: 0.2077 - val_loss: 0.5441 - val_mean_squared_error: 0.2381
Epoch 55/150
5839/5839 [==============================] - 1s 96us/step - loss: 0.5102 - mean_squared_error: 0.2064 - val_loss: 0.5374 - val_mean_squared_error: 0.234

5839/5839 [==============================] - 0s 67us/step - loss: 0.4923 - mean_squared_error: 0.2025 - val_loss: 0.5241 - val_mean_squared_error: 0.2358
Epoch 100/150
5839/5839 [==============================] - 0s 65us/step - loss: 0.4909 - mean_squared_error: 0.2018 - val_loss: 0.5230 - val_mean_squared_error: 0.2350
Epoch 101/150
5839/5839 [==============================] - 0s 63us/step - loss: 0.4889 - mean_squared_error: 0.2019 - val_loss: 0.5198 - val_mean_squared_error: 0.2336
Epoch 102/150
5839/5839 [==============================] - 0s 64us/step - loss: 0.4904 - mean_squared_error: 0.2016 - val_loss: 0.5229 - val_mean_squared_error: 0.2343
Epoch 103/150
5839/5839 [==============================] - 0s 68us/step - loss: 0.4889 - mean_squared_error: 0.2024 - val_loss: 0.5230 - val_mean_squared_error: 0.2371
Epoch 104/150
5839/5839 [==============================] - 0s 70us/step - loss: 0.4887 - mean_squared_error: 0.2023 - val_loss: 0.5220 - val_mean_squared_error: 0.2339
Epoch 

Epoch 148/150
5839/5839 [==============================] - 0s 67us/step - loss: 0.4842 - mean_squared_error: 0.1984 - val_loss: 0.5151 - val_mean_squared_error: 0.2300
Epoch 149/150
5839/5839 [==============================] - 0s 65us/step - loss: 0.4840 - mean_squared_error: 0.1985 - val_loss: 0.5159 - val_mean_squared_error: 0.2299
Epoch 150/150
5839/5839 [==============================] - 0s 64us/step - loss: 0.4842 - mean_squared_error: 0.1990 - val_loss: 0.5139 - val_mean_squared_error: 0.2281
Training MSE: 0.1965
Validation MSE: 0.1669

Training r2: 0.6352
Validation r2: 0.6663


In [12]:
nn4 = models.Sequential()
nn4.add(layers.Dense(128, input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l1(0.02), activation='relu'))
nn4.add(layers.Dense(256, kernel_regularizer=regularizers.l1(0.02), activation='relu'))
nn4.add(layers.Dense(512, kernel_regularizer=regularizers.l1(0.02), activation='relu'))
nn4.add(layers.Dense(1, activation='relu'))

# Compiling the model
nn4.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['mean_squared_error'])

nn4_history = nn4.fit(X_train,
                  y_train,
                  epochs=150,
                  batch_size=256,
                  validation_split = 0.1)

nn_model_evaluation(nn4)

Train on 5839 samples, validate on 649 samples
Epoch 1/150
5839/5839 [==============================] - 1s 142us/step - loss: 180.8595 - mean_squared_error: 8.9929 - val_loss: 147.5954 - val_mean_squared_error: 4.2708
Epoch 2/150
5839/5839 [==============================] - 0s 69us/step - loss: 122.2835 - mean_squared_error: 2.5069 - val_loss: 96.7747 - val_mean_squared_error: 1.4299
Epoch 3/150
5839/5839 [==============================] - 0s 68us/step - loss: 77.6673 - mean_squared_error: 0.9363 - val_loss: 59.0357 - val_mean_squared_error: 0.7073
Epoch 4/150
5839/5839 [==============================] - 0s 73us/step - loss: 46.6240 - mean_squared_error: 0.5052 - val_loss: 35.5319 - val_mean_squared_error: 0.4985
Epoch 5/150
5839/5839 [==============================] - 0s 69us/step - loss: 28.4357 - mean_squared_error: 0.3877 - val_loss: 22.0652 - val_mean_squared_error: 0.4351
Epoch 6/150
5839/5839 [==============================] - 0s 70us/step - loss: 18.0025 - mean_squared_error: 0

Epoch 50/150
5839/5839 [==============================] - 0s 62us/step - loss: 0.7957 - mean_squared_error: 0.2193 - val_loss: 0.8220 - val_mean_squared_error: 0.2506
Epoch 51/150
5839/5839 [==============================] - 0s 60us/step - loss: 0.7904 - mean_squared_error: 0.2200 - val_loss: 0.8182 - val_mean_squared_error: 0.2495
Epoch 52/150
5839/5839 [==============================] - 0s 60us/step - loss: 0.7880 - mean_squared_error: 0.2186 - val_loss: 0.8174 - val_mean_squared_error: 0.2495
Epoch 53/150
5839/5839 [==============================] - 0s 63us/step - loss: 0.7876 - mean_squared_error: 0.2185 - val_loss: 0.8155 - val_mean_squared_error: 0.2461
Epoch 54/150
5839/5839 [==============================] - 0s 62us/step - loss: 0.7907 - mean_squared_error: 0.2190 - val_loss: 0.8184 - val_mean_squared_error: 0.2490
Epoch 55/150
5839/5839 [==============================] - 0s 62us/step - loss: 0.7867 - mean_squared_error: 0.2185 - val_loss: 0.8143 - val_mean_squared_error: 0.248

5839/5839 [==============================] - 0s 67us/step - loss: 0.7509 - mean_squared_error: 0.2152 - val_loss: 0.7789 - val_mean_squared_error: 0.2433
Epoch 100/150
5839/5839 [==============================] - 0s 67us/step - loss: 0.7546 - mean_squared_error: 0.2149 - val_loss: 0.7831 - val_mean_squared_error: 0.2442
Epoch 101/150
5839/5839 [==============================] - 0s 67us/step - loss: 0.7525 - mean_squared_error: 0.2150 - val_loss: 0.7775 - val_mean_squared_error: 0.2413
Epoch 102/150
5839/5839 [==============================] - 0s 66us/step - loss: 0.7498 - mean_squared_error: 0.2141 - val_loss: 0.7863 - val_mean_squared_error: 0.2476
Epoch 103/150
5839/5839 [==============================] - 0s 68us/step - loss: 0.7553 - mean_squared_error: 0.2146 - val_loss: 0.7794 - val_mean_squared_error: 0.2447
Epoch 104/150
5839/5839 [==============================] - ETA: 0s - loss: 0.7485 - mean_squared_error: 0.21 - 0s 66us/step - loss: 0.7516 - mean_squared_error: 0.2146 - val_